In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from metadata_dict import (hca_keys, dcp_to_tier1_mapping)

In [3]:
dcp_path = "./dcp_spreadsheets/"
flat_path = "./flat_dcp/"
tier1_path = "./tier1_output/"
file_name = "ImmuneLandscapeccRCC_metadata_30-01-2023.xlsx"

In [4]:
import ipywidgets as widgets
from IPython.display import display
file_name_dropdown = widgets.Dropdown(
    options= os.listdir(path=dcp_path),
    value=file_name or 'ImmuneLandscapeccRCC_metadata_30-01-2023.xlsx',
    description='Report Entity:',
    disabled=False,
)
display(file_name_dropdown)

Dropdown(description='Report Entity:', index=8, options=('RenalTumorMicroenvironment_metadata_06-04-2023.xlsx'…

In [18]:
file_name = file_name_dropdown.value
print(file_name)

Diabetic Nephropathy snRNA-seq_metadata_01-09-2023.xlsx


In [19]:
spreadsheet = pd.read_excel(os.path.join(dcp_path, file_name), sheet_name=None, skiprows= [0,1,2,4])

## Programmatic names as join keys

In [20]:
# Save all biomaterials and protocols from each tab as join keys

join_keys = {}
for sheet in spreadsheet:
    join_keys[sheet] = []
    for key in spreadsheet[sheet].keys():
        if (key.endswith("biomaterial_id") or key.endswith("protocol_id")) and \
                (not all(spreadsheet[sheet][key].isna())):
            join_keys[sheet].append(key)
    if not join_keys[sheet]:
        del join_keys[sheet]


join_set = set()
for keys in join_keys.values():
    join_set.update(keys)

In [21]:
# Save keys that have double pipe in order to merge metadata together for those keys

keys_with_double_pipe = []
for sheet in join_keys.keys():
    for keys in join_keys[sheet]:
        if any(spreadsheet[sheet][keys].dropna().str.contains("\\|\\|")):
            print(keys, sheet)
            keys_with_double_pipe.append(keys)
keys_with_double_pipe

[]

## Spreadsheet tab order

In [22]:
# default ordering for simple experimental design & protocols
# ordered_sheets = ['Analysis file', 'Cell suspension', 'Specimen from organism', 'Donor organism', 
#                  'Analysis protocol', 'Sequencing protocol', 'Library preparation protocol', 'Dissociation protocol', 'Enrichment protocol', 'Collection protocol']

def field_id_to_tab(value):
    """Convert the programmatic name of a field to their entity/ tab name"""
    return value.split('.')[0].replace('_', ' ').capitalize()


ordered_sheets = []
if 'Analysis file' in join_keys.keys():
    tab = 'Analysis file'
    ordered_sheets.append('Analysis file')
    if 'Sequence file' in join_keys.keys():
        ordered_sheets.append('Sequence file')
else:
    tab = 'Sequence file'
    ordered_sheets.append('Sequence file')


while len(ordered_sheets) < len(join_keys):
    if tab == 'Donor organism':
        break
    for key in join_keys[tab]:
        if field_id_to_tab(key) in ordered_sheets:
            continue
        if key.endswith('protocol_id'):
            ordered_sheets.append(field_id_to_tab(key))
        elif key.endswith('biomaterial_id'):
            tab = field_id_to_tab(key)
            ordered_sheets.append(tab)

ordered_sheets

['Sequence file',
 'Library preparation protocol',
 'Sequencing protocol',
 'Cell suspension',
 'Dissociation protocol',
 'Specimen from organism',
 'Collection protocol',
 'Donor organism']

## Analysis file

In [23]:
if 'Analysis file' in spreadsheet:
    # filter out analysis files that will create duplicates i.e. barcodes and features for same CS and remove analysis_files that did not derive from CS
    spreadsheet['Analysis file'] = spreadsheet['Analysis file'][\
        spreadsheet['Analysis file']['analysis_file.file_core.content_description.ontology'].str.contains('data:3917|data:3112',na=False)]\
        .dropna(subset='cell_suspension.biomaterial_core.biomaterial_id')
    
    # If we have pooled CS for analysis_files, create a new entry for each CS of each file
    analysis_pooledCS = spreadsheet['Analysis file']['cell_suspension.biomaterial_core.biomaterial_id'].str.contains("\\|\\|")
    if any(analysis_pooledCS):
        print(f"analysis files are pooled")
        spreadsheet['Analysis file']['cell_suspension.biomaterial_core.biomaterial_id'] = \
            spreadsheet['Analysis file']['cell_suspension.biomaterial_core.biomaterial_id']\
                .str.split('\\|\\|')\
                .apply(lambda x: x[0] if len(x) == 1 else x)
        spreadsheet['Analysis file'] = spreadsheet['Analysis file'].explode('cell_suspension.biomaterial_core.biomaterial_id')

In [24]:
# edit specific process field names to their Tier 1 field since there are multiple process fields in each sheet

process_fields = {
    'Specimen from organism': {'process.process_core.location': 'sample_collection_site'},
    'Sequence file': {'process.process_core.process_id': 'library_sequencing_run',
                      'process.insdc_experiment.insdc_experiment_accession': 'library_id_repository'}
}

for sheet in process_fields:
    if sheet not in spreadsheet:
        continue
    spreadsheet[sheet] = spreadsheet[sheet].rename(columns=process_fields[sheet])


# Sequence files

In [25]:
def collapse_values(series):
    return ", ".join(series.unique().astype(str))
    
# If we have insdc run accessions, and custom process_id includes this accession, then use insdc run accession instead of process_id
if 'Sequence file' in spreadsheet and \
    'sequence_file.insdc_run_accessions' in spreadsheet['Sequence file'] and \
        spreadsheet['Sequence file']['sequence_file.insdc_run_accessions'].notna().any():
    insdc_in_process = spreadsheet['Sequence file'].apply(lambda row: row['sequence_file.insdc_run_accessions'] in row['library_sequencing_run'], axis=1)
    spreadsheet['Sequence file'].loc[insdc_in_process, 'library_sequencing_run'] = spreadsheet['Sequence file'].loc[insdc_in_process, 'sequence_file.insdc_run_accessions']

# Remove files from library_preparation_protocol that has to do with modality that will not be included in the count_matrix
if 'Sequence file' in spreadsheet:
    print(f"Select the library preparations that you want to include {spreadsheet['Sequence file']['library_preparation_protocol.protocol_core.protocol_id'].unique()}\n")
    if True:
        # Exclude this library_preparation_protocol <SELECT HERE>
        exclude_lib_prep = ['TCR_library_prep']
        spreadsheet['Sequence file'] = spreadsheet['Sequence file'][~spreadsheet['Sequence file']['library_preparation_protocol.protocol_core.protocol_id'].isin(exclude_lib_prep)]
    if False:
        # Include this library_preparation_protocol <OR SELECT HERE>
        include_lib_prep = ['10x_library_prep']
        spreadsheet['Sequence file'] = spreadsheet['Sequence file'][spreadsheet['Sequence file']['library_preparation_protocol.protocol_core.protocol_id'].isin(include_lib_prep)]

    # From sequence file we record everything at the CS level
    spreadsheet['Sequence file'] = spreadsheet['Sequence file']\
        .groupby('cell_suspension.biomaterial_core.biomaterial_id')\
        .agg(collapse_values)\
        .reset_index()


Select the library preparations that you want to include ['library_prep_protocol']



## Initialise output dataframe

In [26]:
# Initiate flat dataframe with analysis_files and unique cell_suspensions (bc for barcode)
if 'Analysis file' in spreadsheet:
    bc = spreadsheet['Analysis file'][['cell_suspension.biomaterial_core.biomaterial_id','analysis_file.file_core.file_name']]\
        .drop_duplicates('cell_suspension.biomaterial_core.biomaterial_id')

# If no analysis files available provide flatten metatadata at the sample level based on the CS from the sequence file tab
# TODO unpool CS for sequence files too
elif 'Sequence file' in spreadsheet:
    bc = spreadsheet['Sequence file'].loc[:, ['cell_suspension.biomaterial_core.biomaterial_id']]\
        .drop_duplicates()

## Merge

In [27]:
# if we have pooled keys, edit the spreadsheet and add the pooled values as extra rows

def process_pooled_keys(sheet, bc, join_key):
    pooled_keys = [key for key in bc[join_key].dropna().unique().tolist() if "||" in key]
    print("We have pooled join keys", pooled_keys)
    for key in pooled_keys:
        unpooled_bool = sheet[join_key].isin(key.split("||"))
        summary_row = {}
        unique_values = {}
        for col, contents in sheet.loc[unpooled_bool].items():
            unique_values[col] = list(contents.dropna().astype('str').unique())
            if len(unique_values[col]) == 1:
                unique_values[col] = unique_values[col][0]
        for col, values in unique_values.items():
            if isinstance(values, list) and len(values) > 1:
                summary_row[col] = '||'.join(values)
            else:
                summary_row[col] = values
        id_key = [name for name in summary_row.keys() if name.endswith("_id")]
        if len(id_key) > 1:
            print(f"Multiple id keys {', '.join(id_key)}")
        summary_row[id_key[0]] = key
        return pd.concat([sheet, pd.DataFrame([summary_row])], ignore_index=True)



In [28]:
# in each sheet merge the flat df with the spreadsheet

for sheet in ordered_sheets:
    if sheet not in spreadsheet.keys():
        continue
    keys_in_tab = spreadsheet[sheet].keys()
    join_key = [key for key in keys_in_tab if key in bc.keys() and key in join_set]
    if len(join_key) > 1:
        print("Multiple join keys in " + sheet + ":\n\t" + "\n\t".join(join_key))
        if sheet in ['Analysis file', 'Sequence file']:
            join_key = 'cell_suspension.biomaterial_core.biomaterial_id'
    else:
        join_key = join_key[0]

    keys_in_tab = [key for key in keys_in_tab if key not in bc.keys() or key == join_key]
    if join_key in keys_with_double_pipe and any(bc[join_key].str.contains("\\|\\|")):
        spreadsheet[sheet] = process_pooled_keys(spreadsheet[sheet], bc, join_key)

    print("Merging sheet " + sheet + " on key " + join_key, sep = "\t")
    bc = bc.merge(spreadsheet[sheet][keys_in_tab], on = join_key, how = "left")
    print("Shape is " + str(bc.shape))
bc

Merging sheet Sequence file on key cell_suspension.biomaterial_core.biomaterial_id
Shape is (6, 18)
Merging sheet Library preparation protocol on key library_preparation_protocol.protocol_core.protocol_id
Shape is (6, 37)
Merging sheet Sequencing protocol on key sequencing_protocol.protocol_core.protocol_id
Shape is (6, 47)
Merging sheet Cell suspension on key cell_suspension.biomaterial_core.biomaterial_id
Shape is (6, 64)
Merging sheet Dissociation protocol on key dissociation_protocol.protocol_core.protocol_id
Shape is (6, 70)
Merging sheet Specimen from organism on key specimen_from_organism.biomaterial_core.biomaterial_id
Shape is (6, 93)
Merging sheet Collection protocol on key collection_protocol.protocol_core.protocol_id
Shape is (6, 98)
Merging sheet Donor organism on key donor_organism.biomaterial_core.biomaterial_id
Shape is (6, 119)


,cell_suspension.biomaterial_core.biomaterial_id,sequence_file.uuid,sequence_file.file_core.file_name,sequence_file.file_core.format,sequence_file.file_core.content_description.text,sequence_file.file_core.content_description.ontology,sequence_file.file_core.content_description.ontology_label,sequence_file.file_core.checksum,sequence_file.read_index,sequence_file.library_prep_id,...,donor_organism.organism_age,donor_organism.development_stage.text,donor_organism.development_stage.ontology,donor_organism.development_stage.ontology_label,donor_organism.diseases.text,donor_organism.diseases.ontology,donor_organism.diseases.ontology_label,donor_organism.medical_history.nutritional_state,donor_organism.medical_history.test_results,donor_organism.medical_history.medication
0,cell_suspension_control_1,"51c04a77-cb8e-492a-83e9-de9c46c37729, 2a9e7377...","R1.fastq.gz, R2.fastq.gz",fastq.gz,DNA sequence (raw),data:3494,DNA sequence,"f9491a935fb91f5463cfd65b6f9203a0, 36d0912fc6e8...","read1, read2",library1,...,54,human adult stage,HsapDv:0000087,human adult stage,Renal mass||Acoustic neuroma||OSA,MONDO:0002367||MONDO:0001569||MONDO:0007147,kidney cancer||acoustic neuroma||obstructive s...,fasting,"sCr 1.28 mg/dL, serum K+ 3.9 mEq/L",NaN
1,cell_suspension_control_2,"336597e4-b3af-4ab9-8425-ea9f1c04799b, f7204ad4...","Norm_S3_L001_R1_001.fastq.gz, Norm_S3_L001_R2_...",fastq.gz,DNA sequence (raw),data:3494,DNA sequence,"31d6d97bf19cc00cfc6313178af554b0, 1fd575225402...","read1, read2, index1",library2,...,62,human adult stage,HsapDv:0000087,human adult stage,Renal mass||Hypertension||AKI (5 months prior)...,MONDO:0002367||MONDO:0001134||MONDO:0006637||M...,kidney cancer||essential hypertension||acute k...,fasting,"sCr 1.21 mg/dL, serum K+ 3.3 mEq/L",NaN
2,cell_suspension_control_3,"9c2c35a2-1400-4f1e-b870-750dbebf6b63, f0e4f00f...","control_S2_L001_R1_001.fastq.gz, control_S2_L0...",fastq.gz,DNA sequence (raw),data:3494,DNA sequence,"8c1ecc2584e5e2f1e07ae97ef0b6d44d, a51d6351b220...","read1, read2, index1",library3,...,61,human adult stage,HsapDv:0000087,human adult stage,Renal mass||anxiety||depression||arthritis||di...,MONDO:0002367||MONDO:0005618||MONDO:0002050||M...,kidney cancer||anxiety disorder||depressive di...,fasting,"sCr 0.89 mg/dL, serum K+ 3.8 mEq/L",NaN
3,cell_suspension_diabetes_1,"e0ca126d-7d50-48b9-bfc3-18a419d92499, 880de2e7...","DN.R1.fastq.gz, DN.R2.fastq.gz",fastq.gz,DNA sequence (raw),data:3494,DNA sequence,"8233b866f89a36e2f6a3b10bf54aba46, 399c187b88aa...","read1, read2",library4,...,74,human adult stage,HsapDv:0000087,human adult stage,right renal mass||type 2 diabetes||adrenal mas...,MONDO:0002367||MONDO:0005148||MONDO:0003924||M...,kidney cancer||type 2 diabetes mellitus||adren...,fasting,"A1c 7.3 mmol/mol, sCr 1.08 mg/dL, serum K+ 4.2...",NaN
4,cell_suspension_diabetes_2,"11e0bb39-9abc-4e8a-906f-73bc5d89e66b, eafbf1ec...","DN_S2_L001_R1_001.fastq.gz, DN_S2_L001_R2_001....",fastq.gz,DNA sequence (raw),data:3494,DNA sequence,"52f6ef9f80ecf909e60672d060444ab8, 34f0c79389fe...","read1, read2, index1",library5,...,52,human adult stage,HsapDv:0000087,human adult stage,Renal mass||Hypertension||type 2 diabetes||pro...,MONDO:0002367||MONDO:0001134||MONDO:0005148||M...,kidney cancer||essential hypertension||type 2 ...,fasting,"A1c 6.7 mmol/mol, sCr 1.26 mg/dL, serum K+ 4.3...","ACE inhibitor, HCTZ"
5,cell_suspension_diabetes_3,"7f495d31-78db-46a8-aa3b-e399913b0118, 8518a4e5...","DN2_S1_L001_R1_001.fastq.gz, DN2_S1_L001_R2_00...",fastq.gz,DNA sequence (raw),data:3494,DNA sequence,"a34bbda366fe4b35c380ef768b8c1def, 7d94c4c2ace8...","read1, read2, index1",library6,...,57,human adult stage,HsapDv:0000087,human adult stage,Renal mass||type 2 diabetes||GERD||hyperlipidemia,MONDO:0002367||MONDO:0005148||MONDO:0007186||M...,kidney cancer||type 2 diabetes mellitus||gastr...,fasting,"A1c 9.7 mmol/mol, sCr 0.7 mg/dL, serum K+ 4.2 ...",NaN


In [29]:
bc.to_csv(os.path.join(flat_path, file_name.replace(".xlsx", "_flat_biomaterial.csv")), index = False)